<a href="https://colab.research.google.com/github/study8677/grpo-rlvr-llm-training/blob/main/grpo-rlvr-llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset
import pandas as pd

# 加载数据集
dataset = load_dataset("litexlang/math23k-litex")

# 打印数据集信息
print(dataset)

# 转换为 DataFrame 查看前几行，确认 Answer 字段
if 'train' in dataset:
    df = dataset['train'].to_pandas()
    display(df.head())
else:
    display(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

math23k.jsonl:   0%|          | 0.00/25.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22329 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'description', 'solution'],
        num_rows: 22329
    })
})


,title,description,solution
0,Math23k_15120,Question: 光华电机厂第二车间计划10天生产135型凤凰相机750架，实际超额(1/...,"claim:\n forall plan, excess, actual R:\n ..."
1,Math23k_15121,Question: 一件原价45元的商品，降价40%后=多少元．\nSolution: 原价...,"claim:\n forall original_price, discount_ra..."
2,Math23k_15122,Question: 一支圆珠笔比一本练习本贵8元，已知练习本的单价是圆珠笔单价的(1/5)．...,"claim:\n forall x, y R:\n x = y + 8\..."
3,Math23k_15123,Question: 计算：7.81*49-78.1*3.8+0.781*90=．\nSolu...,"claim:\n forall a, b, c, result R:\n ..."
4,Math23k_15124,Question: 学校修建围墙，第一天运来砖4560块，比第二天多运960块．两天共运来砖...,"claim:\n forall first_day, second_day, tota..."


# Task
Fine-tune the "Qwen/Qwen2.5-0.5B-Instruct" model using GRPO on the "litexlang/math23k-litex" dataset. First, install `trl` (from source), `peft`, `transformers`, `accelerate`, and `bitsandbytes`. Then, preprocess the dataset by extracting the ground truth numerical answers from the `solution` column and formatting a subset (2k-5k samples) with a `prompt` column (in Qwen-Instruct format) and an `answer` column. Define two reward functions: one to verify the output format (ending with '答案：<number>') and one to check numerical accuracy against the ground truth. Finally, configure and run `GRPOTrainer` with group sampling, and save the fine-tuned model.

## Install Dependencies

### Subtask:
Install the required libraries for GRPO training, including `trl` from source.


**Reasoning**:
Install the required libraries `trl` (from source), `peft`, `transformers`, `accelerate`, and `bitsandbytes` as per the subtask instructions.



In [3]:
!pip install git+https://github.com/huggingface/trl.git
!pip install peft transformers accelerate bitsandbytes

  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-rjhveiyq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-rjhveiyq
  Resolved https://github.com/huggingface/trl.git to commit 5e6ceed0a09e17d50af1ec022d4818b633308dc9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.27.0.dev0-py3-none-any.whl size=522382 sha256=b0d3852b445353ca1c832c4aff2b177bd78bcaeaac393a5b6bb6496f609c4c4f
  Stored in directory: /tmp/pip-ephem-wheel-cache-xh3n707h/wheels/0e/8f/95/dfd1c9271445f7e7e2fcfd9dfdcc8fabf9adc68edd4f2ea5fd
Successfully built trl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 43.7 MB/s eta 0:00:00


## Inspect & Prepare Data

### Subtask:
Load the dataset, extract questions and ground truth answers, and format a subset of 4,000 samples with `prompt` and `answer` columns for GRPO training.


**Reasoning**:
Inspect the first entry of the dataset to understand the exact string structure for parsing the question and answer.



In [4]:
print(dataset['train'][0])
print(dataset['train'][0]['description'])

{'title': 'Math23k_15120', 'description': 'Question: 光华电机厂第二车间计划10天生产135型凤凰相机750架，实际超额(1/5)，实际生产了多少架．\nSolution: 光华电机厂第二车间计划10天生产135型凤凰相机750架，实际超额了$\\frac{1}{5}$。\n\n首先，我们计算超额生产的相机数量：\n\\[ 750 \\times \\frac{1}{5} = 150 \\]\n\n然后，我们将计划生产的相机数量和超额生产的相机数量相加，得到实际生产的相机总数：\n\\[ 750 + 150 = 900 \\]\n\n因此，实际生产了$\\boxed{900}$架相机。\nAnswer: 900', 'solution': 'claim:\n    forall plan, excess, actual R:\n        plan = 750\n        excess = plan * (1 / 5)\n        actual = plan + excess\n        =>:\n            actual = 900\n    prove:\n        excess = 750 * (1 / 5) = 150\n        actual = 750 + 150 = 900'}
Question: 光华电机厂第二车间计划10天生产135型凤凰相机750架，实际超额(1/5)，实际生产了多少架．
Solution: 光华电机厂第二车间计划10天生产135型凤凰相机750架，实际超额了$\frac{1}{5}$。

首先，我们计算超额生产的相机数量：
\[ 750 \times \frac{1}{5} = 150 \]

然后，我们将计划生产的相机数量和超额生产的相机数量相加，得到实际生产的相机总数：
\[ 750 + 150 = 900 \]

因此，实际生产了$\boxed{900}$架相机。
Answer: 900


**Reasoning**:
I will process the dataset to extract questions and answers. Based on the inspection, the `description` column contains the question, solution text, and the final answer labeled as 'Answer: <number>'. Although the instructions mention using the `solution` column for the answer, the `description` column explicitly provides the answer in a structured format ('Answer: 900'). I will parse the 'Answer:' field from the `description` column as it is more reliable than extracting numbers from the formal proof in the `solution` column. I will also format the data for Qwen-Instruct with the specified system prompt.



In [5]:
import re
import random

# System prompt as specified
SYSTEM_PROMPT = "You are a helpful assistant. Solve the math problem and put the final numerical answer at the end in the format: '答案：<number>'."

def process_sample(example):
    # Extract text from description
    desc = example['description']

    # Extract Question
    try:
        question_part = desc.split('Question:')[1].split('Solution:')[0].strip()
    except IndexError:
        # Fallback if format is unexpected
        question_part = desc

    # Extract Answer
    # Priority: Parse 'Answer:' from description
    answer_match = re.search(r'Answer:\s*([\d\.]+)', desc)
    if answer_match:
        answer = answer_match.group(1)
    else:
        # Fallback: Extract the last number from the 'solution' logic or description
        # This handles cases where 'Answer:' might be missing
        nums = re.findall(r"\d+\.?\d*", example['solution'])
        answer = nums[-1] if nums else "0"

    # Construct Prompt
    prompt = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question_part}
    ]

    return {
        'prompt': prompt,
        'answer': answer
    }

# Select a subset of 4000 samples
subset_indices = range(4000)
train_subset = dataset['train'].select(subset_indices)

# Apply processing
processed_dataset = train_subset.map(process_sample)

# Display the first 2 examples to verify
print("First processed example:")
print(processed_dataset[0])
print("\nSecond processed example:")
print(processed_dataset[1])

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

First processed example:
{'title': 'Math23k_15120', 'description': 'Question: 光华电机厂第二车间计划10天生产135型凤凰相机750架，实际超额(1/5)，实际生产了多少架．\nSolution: 光华电机厂第二车间计划10天生产135型凤凰相机750架，实际超额了$\\frac{1}{5}$。\n\n首先，我们计算超额生产的相机数量：\n\\[ 750 \\times \\frac{1}{5} = 150 \\]\n\n然后，我们将计划生产的相机数量和超额生产的相机数量相加，得到实际生产的相机总数：\n\\[ 750 + 150 = 900 \\]\n\n因此，实际生产了$\\boxed{900}$架相机。\nAnswer: 900', 'solution': 'claim:\n    forall plan, excess, actual R:\n        plan = 750\n        excess = plan * (1 / 5)\n        actual = plan + excess\n        =>:\n            actual = 900\n    prove:\n        excess = 750 * (1 / 5) = 150\n        actual = 750 + 150 = 900', 'prompt': [{'content': "You are a helpful assistant. Solve the math problem and put the final numerical answer at the end in the format: '答案：<number>'.", 'role': 'system'}, {'content': '光华电机厂第二车间计划10天生产135型凤凰相机750架，实际超额(1/5)，实际生产了多少架．', 'role': 'user'}], 'answer': '900'}

Second processed example:
{'title': 'Math23k_15121', 'description': 'Question: 一件原价45元的商品，降价40%后=多

## Define Reward Functions

### Subtask:
Define and test the reward functions for format compliance and numerical accuracy.


**Reasoning**:
Define the reward functions `format_reward_func` and `accuracy_reward_func` as specified, and verify them with a test case.



In [6]:
import re

def format_reward_func(completions, **kwargs):
    """Reward function to check if the completion ends with the correct format '答案：<number>'."""
    rewards = []
    pattern = r"答案：\s*([\d\.]+)\s*$"

    for completion in completions:
        # Check if the completion ends with the pattern
        # using search to find it, and ensuring it's at the end
        match = re.search(pattern, completion)
        if match:
            rewards.append(1.0)
        else:
            rewards.append(0.0)
    return rewards

def accuracy_reward_func(completions, answer, **kwargs):
    """Reward function to check if the extracted number matches the ground truth."""
    rewards = []
    pattern = r"答案：\s*([\d\.]+)\s*$"

    for completion, gt in zip(completions, answer):
        match = re.search(pattern, completion)
        if match:
            try:
                extracted_val = float(match.group(1))
                gt_val = float(gt)
                # Check for approximate equality
                if abs(extracted_val - gt_val) < 1e-6:
                    rewards.append(1.0)
                else:
                    rewards.append(0.0)
            except ValueError:
                rewards.append(0.0)
        else:
            rewards.append(0.0)
    return rewards

# Test the reward functions
test_completions = [
    "The solution is simple. 答案：900",       # Correct format and value (assuming gt is 900)
    "The answer is 答案：800",               # Correct format, wrong value
    "The answer is 900",                     # Incorrect format
    "Some text 答案：900 extra text"         # Incorrect format (not at end, though regex might capture if not anchored carefully, but here we enforce it usually)
]
test_answers = ["900", "900", "900", "900"]

print("Format Rewards:", format_reward_func(test_completions))
print("Accuracy Rewards:", accuracy_reward_func(test_completions, answer=test_answers))

Format Rewards: [1.0, 1.0, 0.0, 0.0]
Accuracy Rewards: [1.0, 0.0, 0.0, 0.0]


## Run GRPO Training

### Subtask:
Load the model with quantization, apply chat template formatting to the prompts, configure GRPO with LoRA, and execute the training process.


**Reasoning**:
Load the model, tokenizer, and configuration, preprocess the dataset by applying the chat template, and then execute the GRPO training as specified in the instructions.



# Task
Fine-tune the "Qwen/Qwen2.5-0.5B-Instruct" model using GRPO on a 500-sample subset of the "litexlang/math23k-litex" dataset to ensure quick completion. Preprocess the data to filter prompts exceeding the token limit if necessary. Load the model in `bfloat16` (no quantization) with `flash_attention_2` enabled for speed. Configure `GRPOTrainer` with `max_steps=250`, `num_generations=4`, and a suitable batch size (e.g., 4 or 8). Use the previously defined reward functions (`format_reward_func` and `accuracy_reward_func`) and save the fine-tuned model.

# Plan:
* **Prepare Smaller Dataset**: Select a smaller subset of the dataset (e.g., 500 samples) to ensure the training loop completes quickly.
* **Run Optimized GRPO**: Configure GRPOTrainer with optimized settings: load model in `bfloat16` (no quantization), enable `flash_attention_2`, increase batch size to 8, set `num_generations=4`, and limit `max_steps` to approx 200-300. Execute training.
* **Final Task**: Summarize the fast training run.

## Prepare Smaller Dataset

### Subtask:
Select a subset of 500 samples, apply the chat template, and filter out prompts exceeding 512 tokens.


**Reasoning**:
Load the tokenizer, format the dataset using the chat template, calculate token lengths, filter for prompts under 512 tokens, and select a subset of 500 samples as requested.



In [8]:
from transformers import AutoTokenizer

# 1. Load Tokenizer
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Apply chat template and calculate length
def process_prompt_and_length(example):
    # Apply chat template to generate the full prompt text
    prompt_text = tokenizer.apply_chat_template(
        example['prompt'],
        tokenize=False,
        add_generation_prompt=True
    )
    # Tokenize to get length
    tokens = tokenizer(prompt_text, add_special_tokens=False)['input_ids']
    return {
        'prompt': prompt_text,  # Overwrite 'prompt' with the formatted text string
        'prompt_len': len(tokens)
    }

# Apply transformation to the previously processed dataset
# Note: processed_dataset was created in a previous step
dataset_with_len = processed_dataset.map(process_prompt_and_length)

# 3. Filter prompts > 512 tokens
filtered_dataset = dataset_with_len.filter(lambda x: x['prompt_len'] <= 512)

# 4. Select a subset of 500 samples
small_dataset = filtered_dataset.select(range(500))

print(f"Original size: {len(processed_dataset)}")
print(f"Filtered size (<=512 tokens): {len(filtered_dataset)}")
print(f"Final small dataset size: {len(small_dataset)}")

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4000 [00:00<?, ? examples/s]

Original size: 4000
Filtered size (<=512 tokens): 4000
Final small dataset size: 500


**Reasoning**:
Configure and run the GRPOTrainer with the specified settings: bfloat16, Flash Attention 2 (with fallback), batch size 8, 250 steps, and 4 generations. Use the prepared small_dataset and reward functions.



In [9]:
import torch
from trl import GRPOConfig, GRPOTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

# Reuse model name
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

# 1. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 2. Load Model
# Attempt to use Flash Attention 2 and bfloat16 as requested
print("Loading model...")
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        attn_implementation="flash_attention_2",
        device_map="auto"
    )
    print("Loaded model with bfloat16 and Flash Attention 2.")
except Exception as e:
    print(f"Failed to load with Flash Attention 2: {e}")
    print("Falling back to bfloat16 with default attention (Note: BF16 on T4 might be slow or simulated)...")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )

# 3. LoRA Configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
    bias="none",
    lora_dropout=0.05
)

# 4. GRPO Configuration
training_args = GRPOConfig(
    output_dir="output_grpo_fast",
    learning_rate=2e-5,
    max_steps=250,
    num_generations=4,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    logging_steps=10,
    report_to="none",
    bf16=True, # Request bf16 training
    max_prompt_length=512,
    max_completion_length=200,
)

# 5. Initialize Trainer
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[format_reward_func, accuracy_reward_func],
    args=training_args,
    train_dataset=small_dataset,
    peft_config=peft_config,
    processing_class=tokenizer,
)

# 6. Train
print("Starting GRPO training on subset...")
trainer.train()

# 7. Save Model
trainer.save_model("output_grpo_fast/final_model")
print("Training finished and model saved.")

Loading model...


`torch_dtype` is deprecated! Use `dtype` instead!


Failed to load with Flash Attention 2: FlashAttention2 has been toggled on, but it cannot be used due to the following error: the package flash_attn seems to be not installed. Please refer to the documentation of https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2 to install Flash Attention 2.
Falling back to bfloat16 with default attention (Note: BF16 on T4 might be slow or simulated)...


<string>:192: FutureWarning: The `max_prompt_length` argument is deprecated and will be removed in version 0.28.0. You should instead filter your dataset before training to ensure that prompts do not exceed your desired length.
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Starting GRPO training on subset...


Step,Training Loss
10,0.015900
20,0.005700
30,-0.047600
40,0.068200
50,0.007500
60,0.075900
70,-0.005900
80,0.035400
90,0.091600
100,0.007200


Training finished and model saved.


**Reasoning**:
The previous execution flagged a deprecated `max_prompt_length` argument in `GRPOConfig` and raised an error regarding Flash Attention 2 availability (though handled by fallback). I will remove the deprecated argument (as the dataset is already filtered), remove the Flash Attention 2 request to clean up the execution, and ensure the model loads directly in `bfloat16`.



In [ ]:
import torch
from trl import GRPOConfig, GRPOTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

# Reuse model name
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

# 1. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 2. Load Model
# Use bfloat16 directly. Skipping Flash Attention 2 explicitly to avoid errors since it's not installed.
print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# 3. LoRA Configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
    bias="none",
    lora_dropout=0.05
)

# 4. GRPO Configuration
# Removed max_prompt_length to avoid deprecation warning (dataset is already filtered)
training_args = GRPOConfig(
    output_dir="output_grpo_fast",
    learning_rate=2e-5,
    max_steps=250,
    num_generations=4,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    logging_steps=10,
    report_to="none",
    bf16=True,
    max_completion_length=200,
)

# 5. Initialize Trainer
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[format_reward_func, accuracy_reward_func],
    args=training_args,
    train_dataset=small_dataset,
    peft_config=peft_config,
    processing_class=tokenizer,
)

# 6. Train
print("Starting GRPO training on subset...")
trainer.train()

# 7. Save Model
trainer.save_model("output_grpo_fast/final_model")
print("Training finished and model saved.")

Loading model...


The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Starting GRPO training on subset...


Step,Training Loss
10,0.123100
20,0.094500
30,0.013600
40,0.079400
50,-0.006200
60,-0.003100
70,0.062600
